## Training/ Testing data

In [88]:
import numpy as np
import pandas as pd
import os
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from time import time
from operator import itemgetter
import time
from scipy import stats
from sklearn.metrics import confusion_matrix
import itertools
import brew
from brew.base import Ensemble
from brew.combination.combiner import Combiner
from brew.stacking.stacker import EnsembleStack
from brew.stacking.stacker import EnsembleStackClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_curve, auc

from sklearn.ensemble import RandomForestClassifier

from sklearn import (preprocessing, metrics, cross_validation)

from sklearn.externals import joblib


In [3]:
#Change name of the location of Dropbox Folder.
server = 'H:/MatlabProjects/Nate'

In [9]:
#upsample minority class

def upsample(data, labels):
    """
    This function is to upsample the data for the under-repressenting class with replacement. 
    The result will be a matrix with the same amount of species for each class.
    """
    
    from collections import defaultdict
    import random

    label_indices = defaultdict(lambda: list())
    for idx, label in enumerate(labels):
        label_indices[label].append(idx)

    largest_class_size = max(map(lambda l: len(l), label_indices.values()))

    upsampled_indices = []
    for label, indices in label_indices.items():
        sampled_indices = indices[:]
        while len(sampled_indices) < largest_class_size:
            sampled_indices.append(random.choice(indices))
        upsampled_indices.extend(sampled_indices)

    upsampled_labels = labels[upsampled_indices]
    upsampled_data = data[upsampled_indices, :]

    return upsampled_data, upsampled_labels

# find optimal cutoff using Youdel index
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Input:
    target values 
    predicted values
    
    Returns
    optimal cutoff value
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

# pretty plot confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



# Train Logistic

In [297]:
def train_logistic(X,Y):
    '''This function perform logistic regression training using cross-validation'''
    start = time.time()

    loo = LeaveOneOut()
    fold = StratifiedKFold(Y, n_folds=10, shuffle=True, random_state=0)
    SP = StratifiedShuffleSplit(n_splits=50, test_size=.2, random_state=0)

    grid = {'penalty':['l2'],'fit_intercept':[True, False],'C': np.power(10.0, np.linspace(-15, 15, 200)), 'solver':['newton-cg','liblinear','lbfgs'],'class_weight':['balanced']}
    #     score = None
    # Using LogisticRegressionCV (cross validation search for Logistic regression) and report ROC_AUC score
    clf = LogisticRegression(random_state=0, tol=.001)
    searchCV = RandomizedSearchCV(clf, grid,n_iter = 1000, scoring='roc_auc', cv=SP, n_jobs = -1)

    searchCV.fit(X, Y)

    cvs_score = searchCV.cv_results_['mean_test_score'][searchCV.best_index_]
    cvs_std = searchCV.cv_results_['std_test_score'][searchCV.best_index_]
    param = searchCV.cv_results_['params'][searchCV.best_index_]

#     print(cvs_score,cvs_std)

    end = time.time()
    print (end - start)

#     grid_scores = searchCV.grid_scores_
#     top_scores_logistic = sorted(grid_scores,
#                         key=itemgetter(1),
#                         reverse=True)[:5]
#     for i, score in enumerate(top_scores_logistic):
#         print("Model with rank: {0}".format(i + 1))
#         print(("Mean validation score: "
#                "{0:.3f} (std: {1:.3f})").format(
#                score.mean_validation_score,
#                np.std(score.cv_validation_scores)))
#         print("Parameters: {0}".format(score.parameters))
#         print("")
    
    return param, cvs_score, cvs_std
    


## Training with Cross Validation and dump model to a .pkl file

In [340]:
from sklearn.externals import joblib

cv_param = []
cv_score = []
cv_std = []
for count in range(1,25):
    X = np.genfromtxt('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/TrainX.csv' %(server,count), delimiter=',')
    Y = np.genfromtxt('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/TrainY.csv'% (server,count), delimiter=',')
    score = train_logistic(X,Y)
    clf = LogisticRegression(**score[0],random_state = 0, tol = .001).fit(X,Y)
    joblib.dump(clf, '%s/Dropbox/Rotation 3 Project/Models/Logistic/hour%02d.pkl' %(server,count)) 
    cv_param.append(score[0])
    cv_score.append(score[1])
    cv_std.append(score[2])
np.savetxt('%s/Dropbox/Rotation 3 Project/Result/Logistic_CV_score.csv' %server, cv_score, delimiter=',')
np.savetxt('%s/Dropbox/Rotation 3 Project/Result/Logistic_CV_std.csv' %server, cv_std, delimiter=',')
np.savetxt('%s/Dropbox/Rotation 3 Project/Result/Logistic_CV_param.txt' %server, cv_param, delimiter=',', fmt='%s')


60.501932859420776
56.084574937820435
70.82893919944763
70.09731769561768
63.98280382156372
63.963582277297974
65.17745089530945
58.21783995628357
61.18360638618469
70.73594307899475
59.10253047943115
61.09845185279846
59.855053186416626
59.92598104476929
63.29697561264038
63.44581174850464


C:\Users\ntran\Anaconda3\lib\site-packages\sklearn\cross_validation.py:552: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


60.86022877693176


C:\Users\ntran\Anaconda3\lib\site-packages\sklearn\cross_validation.py:552: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


71.30455875396729


...

86.31489682197571


C:\Users\ntran\Anaconda3\lib\site-packages\sklearn\cross_validation.py:552: Warning: The least populated class in y has only 7 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


74.09549689292908


C:\Users\ntran\Anaconda3\lib\site-packages\sklearn\cross_validation.py:552: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


76.13296556472778


# Find optimal cutoff using Youden's index and save them for each hour

In [339]:
cut_off = []
for count in range(1,25):
    X = np.genfromtxt('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/TrainX.csv' % (server,count), delimiter=',')
    Y = np.genfromtxt('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/TrainY.csv'% (server,count), delimiter=',')
    clf = joblib.load('%s/Dropbox/Rotation 3 Project/Models/Logistic/hour%02d.pkl' %(server,count)) 
    Y_proba = clf.predict_proba(X)
    threshold = Find_Optimal_Cutoff(Y, Y_proba[:,1])
    cut_off.append(threshold)
np.savetxt('%s/Dropbox/Rotation 3 Project/Result/cut off log.csv' %server, cut_off, delimiter=',')

## Testing with the 20% hold out

In [341]:
roc_auc_score = []
for count in range(1,25):
    X_test1 = pd.read_csv('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/TestX.csv' %(server,count), header = None)
    Y_test1 = pd.read_csv('%s/Dropbox/Rotation 3 Project/TrainTest/%02d/TestY.csv'%(server,count), header = None)

    clf = joblib.load('%s/Dropbox/Rotation 3 Project/Models/Logistic/hour%02d.pkl' %(server,count))
    Y_proba = clf.predict_proba(X_test1)
    Y_predict = clf.predict(X_test1)

#     from sklearn.metrics import confusion_matrix
#     print(confusion_matrix(Y_test1, Y_predict))

    fpr, tpr, thresholds = roc_curve(Y_test1, Y_proba[:,1])
    roc_auc = auc(fpr, tpr)
    roc_auc_score.append(roc_auc)

np.savetxt('%s/Dropbox/Rotation 3 Project/Result/Log_holdouttest_AUCscore.csv' %server, roc_auc_score, delimiter=',')

## Testing with the Columbia patients

In [342]:
roc_auc_score = []
for count in range(1,25):
    datapath = '%s/Dropbox/Rotation 3 Project/Data/ColumbiaTest/featurematrixandtarget_%02dh' %(server,count)
    datacontent = sio.loadmat(datapath)

    X_test2 = datacontent['featm']
    X_test2 = np.nan_to_num(X_test2) # replace nan with 0
    nsubjects = X_test2.shape[0]

    Yv = datacontent['y']
    Y_test2 = np.ravel(Yv)

    clf = joblib.load('%s/Dropbox/Rotation 3 Project/Models/Logistic/hour%02d.pkl' %(server,count))
    Y_proba = clf.predict_proba(X_test2)
    Y_predict = clf.predict(X_test2)

    # from sklearn.metrics import confusion_matrix
    # print(confusion_matrix(Y_test2, Y_predict))

    # fpr, tpr, thresholds = roc_curve(Y_test, Y_df)
    fpr, tpr, thresholds = roc_curve(Y_test2, Y_proba[:,1])
    roc_auc = auc(fpr, tpr)
    roc_auc_score.append(roc_auc)

np.savetxt('%s/Dropbox/Rotation 3 Project/Result/Log_columbiatest_AUCscore.csv' %server, roc_auc_score, delimiter=',')

C:\Users\ntran\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:534: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
C:\Users\ntran\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:96: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):


## Testing with the delayshunt

In [343]:
score = []
for count in range(1,25):
    datapath = '%s/Dropbox/Rotation 3 Project/Data/DelayedShuntTest/featurematrixandtarget_%02dh' %(server,count)
    datacontent = sio.loadmat(datapath)

    X_test3 = datacontent['featm']
    X_test3 = np.nan_to_num(X_test3) # replace nan with 0
    nsubjects = X_test3.shape[0]

    Yv = datacontent['y']
    Y_test3 = np.ravel(Yv)

    clf = joblib.load('%s/Dropbox/Rotation 3 Project/Models/Logistic/hour%02d.pkl' %(server,count))
    Y_proba = clf.predict_proba(X_test3)
    s = clf.score(X_test3, Y_test3)
    score.append(s)

    # from sklearn.metrics import confusion_matrix
    # print(confusion_matrix(Y_test2, Y_predict))

#     roc_auc_score = []
#     # fpr, tpr, thresholds = roc_curve(Y_test, Y_df)
#     fpr, tpr, thresholds = roc_curve(Y_test2, Y_proba[:,1])
#     roc_auc = auc(fpr, tpr)
#     roc_auc_score.append(roc_auc)

np.savetxt('%s/Dropbox/Rotation 3 Project/Result/Log_delayshunt_ACCUscore.csv' %server, score, delimiter=',')
